In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.16.1'

In [6]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        # open file
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield (sample_idx,)

    def __new__(cls):
        tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape=(1,), dtype=tf.int64),
        )
        pass

In [5]:
def train(dataset, num_epochs=2):
    for epoch_num in num_epochs:
        for sample in dataset:
            time.sleep(0.01)

new called
